In [1]:
#Оформление New Обводка таблиц
from IPython.core.display import HTML

HTML("""
<style>
div.jp-MarkdownOutput {font-family: "Times New Roman", Times, serif;}
div.jp-RenderedText.jp-OutputArea-output {border: 0.2px black solid !important; color: black !important;}
table.dataframe td, table.dataframe th {
    border: 0.2px black solid !important; color: black !important;
}
</style>
""")

In [2]:
#Импорт библиотек и создание таблицы "Сводка ЗК"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Импорт таблиц
df1 = pd.read_excel(r"source_transfers.xlsx")
df2 = pd.read_excel(r"source_dbo.xlsx")

In [4]:
#Добавление столбцов для сводки в ДБО
df2['Статус'] = 'Работает'
df2['Есть в ДБО'] = 1

In [5]:
#New Заполнить строчки без ID нулями
#df1["ID получателя"].fillna(0, inplace=True)
df1["ID получателя"].dropna(inplace=True)

In [6]:
#Расчёт комиссии 
df1.loc[df1['Код точки отправителя'] == 'B', '% комиссии'] =
df1.loc[df1['Код точки отправителя'] != 'B', '% комиссии'] = 
df1['Комиссия'] = df1['Сумма перевода'] * df1['% комиссии']

In [7]:
#Сводка переводов и ДБО с удалением дубликатов из ДБО. Если в ДБО нет, подставляем 0.
df3 = pd.merge(df1[['Дата перевода', 'ID получателя', 'Сумма перевода', 'Код точки отправителя', 'Получатель. Наименование', '% комиссии', 'Комиссия']], df2[['Клиент ID', 'Статус', 'Есть в ДБО']].drop_duplicates(subset=['Клиент ID']), how='left', left_on='ID получателя', right_on='Клиент ID')
df3['Есть в ДБО'] = df3['Есть в ДБО'].fillna(0)

In [8]:
#Импорт кредитов, удаление дубликатов, чтобы не брать закрытые кредиты клиентов, когда есть открытые
df4 = pd.read_excel(r"source_credits.xlsx")
df4_upd = df4.sort_values(['ID клиента', 'Дата закрытия']).drop_duplicates(subset=['ID клиента'], keep='last')

In [9]:
#Ссудный счёт начинается на 45511
values = ['45511']
conditions = list(map(df4_upd['Ссудный счет'].str.contains, values))
df4_upd['Ссудный счёт начинается на 45511'] = np.select(conditions, values, 'other')

In [10]:
#Сводка Переводов и ДБО с кредитами
df5 = pd.merge(df3, df4_upd[['ID клиента', 'Вид кредита', 'Покупатель кредита', 'Дата закрытия', 'Ссудный счёт начинается на 45511']], how='left', left_on='ID получателя', right_on='ID клиента')

# Отчёт по ЗК

## Введение

...

Общие показатели за апрель:

In [ ]:
#Вывод общих показателей New Формат чисел
totalcount = df3['Сумма перевода'].count()
print("Количество переводов: {}".format(totalcount))
totalsum = df3['Сумма перевода'].sum().round().astype(int)
print("Сумма переводов: {:,}".format(totalsum))
totalsum = df3['Комиссия'].sum().round().astype(int)
print("Сумма комиссии Банка в пользу Золотой короны: {:,} руб.".format(totalsum)) 

Сумма и количество переводов по месяцам:

In [ ]:
#Monthly New
plt.style.use('seaborn')

x = ['Февраль', 'Март', 'Апрель']
sum_abc = []
num_abc = []

a4_dims = (8, 4)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=a4_dims)

ax1.bar(x, sum_abc)
ax2.bar(x, num_abc)

ax1.ticklabel_format(style='plain', axis='y')

ax1.set_title("Сумма переводов по месяцам")
ax2.set_title("Количество клиентов по месяцам")

plt.tight_layout()
plt.show()

In [13]:
#Объединение с цессиями
#Скопировать свежие файлы цессий X:\Я МЕНЕДЖЕР\ЦЕССИИ, объединить с прошлым, ID из кредитов
cessia = pd.read_excel(r"source_cessia.xlsx")
cessia['Цессия'] = 'Да'
df6 = pd.merge(df5, cessia[['ID', 'Цессия']], how='left', left_on='ID получателя', right_on='ID')

In [14]:
#ABC создание файла Эксель NEW
df_abc = df6[['ID получателя', 'Код точки отправителя', 'Получатель. Наименование', 'Комиссия', 'Сумма перевода']]
ids = df_abc.groupby(['Получатель. Наименование', 'Код точки отправителя']).agg({"Комиссия": "sum", "Сумма перевода": "sum"}).reset_index()
abc2 = ids[ids['Получатель. Наименование'].duplicated(keep=False)]
#abc.to_excel(r'result_abc.xlsx')

#ABC создание файла Эксель OLD
#ids = df6['Получатель. Наименование']
#abc = df6[ids.isin(ids[ids.duplicated()])]
#abc2 = abc.drop_duplicates(subset=['Получатель. Наименование', 'Код точки отправителя'], keep=False)
#abc2_pivot = pd.pivot_table(abc2, index = ['ID получателя', 'Получатель. Наименование'], columns = 'Код точки отправителя', values = ['Сумма перевода', 'Комиссия'], aggfunc = ['sum'])
#abc2_pivot.to_excel(r'result_abc2.xlsx')

Показатели по клиентам, которые используют несколько способов платежа:

In [ ]:
#ABC печать в отчёт NEW
abc2total = abc2['Сумма перевода'].sum()
abc2totalcom = abc2['Комиссия'].sum()
abc2totalcount = abc2['Получатель. Наименование'].nunique()
abc2series = {'Показатель': ['Сумма перевода','Комиссия','Количество клиентов'],
        'Сумма за апрель': [abc2total,abc2totalcom,abc2totalcount]
        }
abc2df = pd.DataFrame(abc2series, columns = ['Показатель', 'Сумма за апрель'])
abc2df["Сумма за апрель"] = abc2df['Сумма за апрель'].round().astype(int).apply(lambda x : "{:,}".format(x))
print(abc2df)

In [16]:
#Убираем ABC из списка переводов
df7=pd.merge(df6, abc2['Получатель. Наименование'], how='left', suffixes=('','_y'), indicator=True)
df8 = df7[df7['_merge']=='left_only'][df6.columns]

## Анализ клиентов

## 1.	Клиенты, которые использовали систему переводов НЕ для погашения кредитов
Предполагаемое действие:  закрыть данную возможность, ввести доп. комиссию, оставить как есть.

In [17]:
#Не кредиты. Для выгрузки.
nocredits = df8[df8['Вид кредита'].isnull()]
nocredits_pivot = pd.pivot_table(nocredits, index = ['ID получателя', 'Получатель. Наименование'], columns = 'Код точки отправителя', values = ['Сумма перевода','Комиссия'], aggfunc = ['sum']) 
nocredits_pivot.to_excel(r'result_not_credit.xlsx')

Показатели по клиентам, которые переводят деньги не за кредит:

In [ ]:
#Не кредит. Для отчёта.
nocredits_count = nocredits['ID получателя'].nunique()
print("Кол-во, шт: {}".format(nocredits_count))
nocredits_totalsum = nocredits['Сумма перевода'].sum().astype(int)
print("Сумма переводов, руб.: {:,}".format(nocredits_totalsum))
nocredits_comsum = nocredits['Комиссия'].sum().astype(int)
print("Сумма кв, руб.: {:,}".format(nocredits_comsum))

nocredits_table = nocredits[['ID получателя', 'Код точки отправителя', 'Сумма перевода', 'Комиссия']].groupby(['Код точки отправителя']).agg({
    'ID получателя': ['nunique'],
    'Сумма перевода': ['sum'],
    'Комиссия': ['sum']
})
nocredits_table["Сумма перевода"] = nocredits_table['Сумма перевода'].astype(float).astype(int)
nocredits_table["Комиссия"] = nocredits_table['Комиссия'].astype(float).astype(int)
print(nocredits_table)


Список клиентов представлен в документе result_not_credit.xlsx.

Сумма и количество переводов по месяцам:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_nocredit.xlsx")

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество клиентов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

...

## 2.	Клиенты, которые использовали систему переводов для погашения кредитов

2.1.	Ниже приведена сводка данных по всем клиентам, которые использовали Золотую корону для оплаты кредитов. В файле result_credits.xlsx находится список клиентов.

In [20]:
#Кредиты для выгрузки
credits = df8[df8['Вид кредита'].notnull()]
credits_pivot = pd.pivot_table(credits, index = ['ID получателя', 'Получатель. Наименование'], columns = 'Код точки отправителя', values = ['Сумма перевода','Комиссия'], aggfunc = ['sum']) 
credits_pivot.to_excel(r'result_credits.xlsx')

In [ ]:
#Кредиты для отчёта
credits_report = pd.pivot_table(credits, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False
                                , margins=True, margins_name = 'Total'
                               ).stack().round(1)
print(credits_report)

% соотношение:

In [ ]:
#% соотношение
credits_report_perc = credits_report.drop('Total', level='Код точки отправителя')
credits_report_perc2 = credits_report_perc['ID получателя'].groupby(['Есть в ДБО']).apply(lambda g: g / g.sum()).round(2)
print(credits_report_perc2)

Сумма переводов и количество клиентов по месяцам по всем, кто переводил за кредиты:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_allcredits.xlsx")

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum', 'csum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum', 'cnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество клиентов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

Сумма переводов и количество клиентов по месяцам по всем, кто переводил за кредиты и есть в списке ДБО:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_allcredits_dbo_1.xlsx")

#x = np.arange(len(month))

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum', 'csum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum', 'cnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество переводов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

Сумма переводов и количество клиентов по месяцам по всем, кто переводил за кредиты и отсутствует в списке ДБО:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_allcredits_dbo_0.xlsx")

#x = np.arange(len(month))

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum', 'csum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum', 'cnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество переводов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

...

## Промежуточный итог

Увеличивается количество и сумма переводов клиентов, которые переводят средства через мобильное приложение. Выведем в процетах изменение сумм переводов и их количества по месяцам по всем категориям:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_allcredits_new.xlsx")
a4_dims = (16, 8)
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=a4_dims)

data.plot(x ='month', y = ['% сайта по сумме', '% сайта по количеству'], kind = 'bar', ax=ax1)
data.plot(x = 'month', y = ['% офлайна по сумме', '% офлайна по количеству'], kind = 'bar', ax=ax2)
data.plot(x ='month', y = ['% приложения по сумме', '% приложения по количеству'], kind = 'bar', ax=ax3)

ax1.legend()
ax1.set_title("Сайт")
ax1.set_xlabel('Месяц')
ax1.set_ylabel('Процент')

ax2.legend()
ax2.set_title("Офлайн")
ax2.set_xlabel('Месяц')
ax2.set_ylabel('Процент')
ax2.ticklabel_format(style='plain', axis='y')

ax3.legend()
ax3.set_title("Приложение")
ax3.set_xlabel('Месяц')
ax3.set_ylabel('Процент')

plt.tight_layout()
plt.show()

...

## 2.2. Клиенты с закрытыми кредитными договорами (предполагаем это проданные кредиты) 
Анализируем в разрезе способа осуществления перевода для того, чтобы увидеть объем таких платежей и их распределение по способу перевода:

In [120]:
#Цессии. Кредиты с не пустыми полями 'Дата закрытия' или 'Ссудный счёт начинается на 45511' или 'Покупатель кредита'
cessia_f = credits[(credits['Дата закрытия'].notnull()) | (credits['Ссудный счёт начинается на 45511'] != 'other') | (credits['Покупатель кредита'].notnull())]
cessia_pivot = pd.pivot_table(cessia_f, index = ['ID получателя', 'Получатель. Наименование'], columns = 'Код точки отправителя', values = ['Сумма перевода','Комиссия'], aggfunc = ['sum']) 
cessia_pivot.to_excel(r'result_cessia.xlsx')

In [ ]:
#Кредиты для отчёта
cessia_report = pd.pivot_table(cessia_f, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False, margins=True, margins_name = 'Total').stack().round()
#cessia_report['% ID получателя'] = (cessia_report['ID получателя']/cessia_report['ID получателя'].sum())*100
print(cessia_report)

% соотношение:

In [ ]:
#% соотношение
print('% соотношение')
cessia_report_perc = cessia_report.drop('Total', level='Код точки отправителя')
cessia_report_perc2 = cessia_report_perc['ID получателя'].groupby(['Есть в ДБО']).apply(lambda g: g / g.sum()).round(2)
print(cessia_report_perc2)

Сумма переводов и количество клиентов по месяцам по всем, кто переводил за кредиты и находится в списках проданных:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_credits_cessia.xlsx")

#x = np.arange(len(month))

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum', 'csum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum', 'cnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество клиентов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

...

## 2.3.	Клиенты с действующими кредитными договорами в разрезе наличия у них договора ДБО

In [125]:
#Кредиты открытые. Кредиты с пустыми полями 'Дата закрытия' и 'Ссудный счёт начинается на 45511' и 'Покупатель кредита'.
open_f = credits[(credits['Дата закрытия'].isnull()) & (credits['Ссудный счёт начинается на 45511'] == 'other') & (credits['Покупатель кредита'].isnull())]
open_pivot = pd.pivot_table(open_f, index = ['ID получателя', 'Получатель. Наименование'], columns = 'Код точки отправителя', values = ['Сумма перевода','Комиссия'], aggfunc = ['sum']) 
open_pivot.to_excel(r'result_open_credits.xlsx')

In [ ]:
#Таблица для отчёта
open_report = pd.pivot_table(open_f, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False, margins=True, margins_name = 'Total').stack().round()
print(open_report)

% соотношение:

In [ ]:
#% соотношение
open_report_perc = open_report.drop('Total', level='Код точки отправителя')
open_report_perc2 = open_report_perc['ID получателя'].groupby(['Есть в ДБО']).apply(lambda g: g / g.sum())
print(open_report_perc2)

Сумма переводов и количество клиентов по месяцам по всем, кто переводил за кредиты и является действующим клиентом:

In [ ]:
plt.style.use('seaborn')

data = pd.read_excel(r"month_credits_open.xlsx")

#x = np.arange(len(month))

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

data.plot(x = 'month', y = ['allsum', 'asum', 'bsum', 'csum'], kind = 'bar', ax=ax1)
data.plot(x ='month', y = ['allnum', 'anum', 'bnum', 'cnum'], kind = 'bar', ax=ax2)

ax1.legend()
ax1.set_title("Cумма переводов по месяцам")
ax1.set_xlabel('month')
ax1.set_ylabel('Sum')
ax1.ticklabel_format(style='plain', axis='y')

ax2.legend()
ax2.set_title("Количество переводов по месяцам")
ax2.set_xlabel('month')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

...





## Рассмотрим более подробно клиентов, у которых имеются действующие кредитные договоры и их способы перевода:

2.3.1.	Клиенты платят офлайн, но при этом имеют договор ДБО в состоянии «Работает»

Предполагаемое действие: отправить смс о возможности платить через МБ 


In [ ]:
#Клиенты платят офлайн, но при этом имеют договор ДБО в состоянии «Работает»
open_b_f = open_f[(open_f['Код точки отправителя'] == 'B') & (open_f['Есть в ДБО'] == 1)]
open_b_f.to_excel(r'result_credits_offline_dbo_1.xlsx')
open_b = pd.pivot_table(open_b_f, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False).stack().round()
print(open_b)

...

2.3.2.	Клиенты платят через сайт, но имеющие договор ДБО в состоянии «Работает»

Предполагаемое действие: смс о возможности платить через МБ 

In [ ]:
#Клиенты платят через сайт, но имеющие договор ДБО в состоянии «Работает»
open_a_f = open_f[(open_f['Код точки отправителя'] == 'A') & (open_f['Есть в ДБО'] == 1)]
open_a_f.to_excel(r'result_credits_website_dbo_1.xlsx')
open_a = pd.pivot_table(open_a_f, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False).stack().round()
print(open_a)

...

2.3.3.	Клиенты платят офлайн и не имеют действующего договора ДБО

Предполагаемое действие: звонок, агитация и информация о возможности платить через МБ, подключение ДБО. Если от клиента отказ, настоятельная рекомендация платить через сайт

In [ ]:
#Клиенты платят офлайн и не имеют действующего договора ДБО
open_b_f_bezdbo = open_f[(open_f['Код точки отправителя'] == 'B') & (open_f['Есть в ДБО'] == 0)]
open_b_f_bezdbo.to_excel(r'result_credits_offline_dbo_0.xlsx')
open_b_bezdbo = pd.pivot_table(open_b_f_bezdbo, index='Есть в ДБО',  
                            values=['ID получателя','Сумма перевода','Комиссия'],
                            columns='Код точки отправителя',
                            aggfunc= {'ID получателя':'nunique', 'Комиссия':sum, 'Сумма перевода':sum }, 
                            fill_value=0, dropna=False).stack().round()
print(open_b_bezdbo)

...
